# Packing unit cubes

In [ ]:
#@title Data and verification

import numpy as np

# --- Core Geometric Functions ---

def rotation_matrix_3d(rx_deg, ry_deg, rz_deg):
    """
    Generates a 3D rotation matrix from Euler angles in degrees.
    The order of rotation is Z, then Y, then X.
    """
    # Convert degrees to radians for numpy's trigonometric functions
    rx, ry, rz = np.deg2rad([rx_deg, ry_deg, rz_deg])

    # Pre-calculate cosines and sines
    cos_x, sin_x = np.cos(rx), np.sin(rx)
    cos_y, sin_y = np.cos(ry), np.sin(ry)
    cos_z, sin_z = np.cos(rz), np.sin(rz)

    # Define rotation matrices for each axis
    rot_x = np.array([[1, 0, 0], [0, cos_x, -sin_x], [0, sin_x, cos_x]])
    rot_y = np.array([[cos_y, 0, sin_y], [0, 1, 0], [-sin_y, 0, cos_y]])
    rot_z = np.array([[cos_z, -sin_z, 0], [sin_z, cos_z, 0], [0, 0, 1]])

    # Combine rotations: R = Rz * Ry * Rx
    return rot_z @ rot_y @ rot_x

class Cube:
    """A class to represent a single cube in 3D space."""
    def __init__(self, center, rotation_angles, side_length=1.0):
        """
        Initializes a Cube object.
        Args:
            center (list or np.array): The [x, y, z] coordinates of the cube's center.
            rotation_angles (list or np.array): The [rx, ry, rz] rotation angles in degrees.
            side_length (float): The side length of the cube.
        """
        self.center = np.array(center)
        self.rot_mat = rotation_matrix_3d(*rotation_angles)

        # The local axes of the cube (its face normals) are the columns of the rotation matrix.
        self.axes = [self.rot_mat[:, i] for i in range(3)]

        # Calculate the world coordinates of the 8 vertices
        half_side = side_length / 2.0
        local_verts = np.array([
            [-1, -1, -1], [-1, -1, 1], [-1, 1, -1], [-1, 1, 1],
            [1, -1, -1], [1, -1, 1], [1, 1, -1], [1, 1, 1]
        ]) * half_side
        # Rotate and translate the vertices to their world positions
        self.vertices = (local_verts @ self.rot_mat.T) + self.center

# --- Separating Axis Theorem (SAT) Implementation ---

def _project_on_axis(cube, axis):
    """Projects a cube's vertices onto an axis to find the min/max interval."""
    # Ensure axis is a unit vector for consistent projection values
    axis_norm = np.linalg.norm(axis)
    if axis_norm < 1e-9: # Avoid division by zero for zero-length cross products
        return 0, 0
    axis = axis / axis_norm

    # Project each vertex onto the axis
    projections = [np.dot(vertex, axis) for vertex in cube.vertices]
    return min(projections), max(projections)

def check_intersection(cube1, cube2):
    """
    Checks for intersection between two cubes using the Separating Axis Theorem (SAT).

    The SAT states that two convex objects do not intersect if there exists an axis
    (a "separating axis") onto which their projections do not overlap. For two cubes,
    we only need to test 15 potential separating axes.

    Returns:
        True if the cubes intersect, False otherwise.
    """
    # The 15 axes to test are:
    # 1. The 3 face normals of Cube 1
    # 2. The 3 face normals of Cube 2
    # 3. The 9 cross products of each edge from Cube 1 with each edge from Cube 2
    #    (Since cube edges are parallel to its axes, we use the axes).
    potential_axes = cube1.axes + cube2.axes
    for axis1 in cube1.axes:
        for axis2 in cube2.axes:
            potential_axes.append(np.cross(axis1, axis2))

    for axis in potential_axes:
        # Project both cubes onto the current axis
        min1, max1 = _project_on_axis(cube1, axis)
        min2, max2 = _project_on_axis(cube2, axis)

        # Check if the projections (intervals) overlap.
        # If they DON'T overlap, we have found a separating axis.
        if max1 < min2 or max2 < min1:
            return False # The cubes are separated, no intersection.

    # If we tested all 15 axes and found an overlap on every single one,
    # the cubes must be intersecting.
    return True

# --- Bounding Box Calculation ---

def calculate_bounding_box_side(cubes):
    """
    Calculates the side length of the smallest axis-aligned bounding cube
    that contains all the given cubes.
    """
    if not cubes:
        return 0

    # Collect all vertices from all cubes into a single array
    all_vertices = np.vstack([cube.vertices for cube in cubes])

    # Find the minimum and maximum coordinates across all vertices
    min_coords = np.min(all_vertices, axis=0) # [min_x, min_y, min_z]
    max_coords = np.max(all_vertices, axis=0) # [max_x, max_y, max_z]

    # Calculate the side lengths of the bounding box along each axis
    span = max_coords - min_coords # [span_x, span_y, span_z]

    # The side of the smallest *cube* that bounds this box is the largest of the spans
    return np.max(span)

# --- Main Analysis Function ---

def analyze_cube_packing(placements):
    """
    Main function to perform the analysis. It checks for intersections and, if none
    are found, calculates the bounding box side length.

    Args:
        placements (np.array): An Nx6 numpy array where each row is
                               [cx, cy, cz, rx, ry, rz] for a cube.
    """
    print("--- Starting Cube Packing Analysis ---")
    num_cubes = len(placements)
    if num_cubes < 2:
        print("Analysis requires at least two cubes.")
        return

    # Create Cube objects from the raw placement data
    cubes = [Cube(p[:3], p[3:]) for p in placements]
    print(f"Successfully created {len(cubes)} cube objects.")

    # 1. Check for intersections by comparing every unique pair of cubes
    print("\n1. Checking for intersections...")
    intersection_found = False
    for i in range(num_cubes):
        for j in range(i + 1, num_cubes):
            if check_intersection(cubes[i], cubes[j]):
                print(f"   -> ❌ INTERSECTION DETECTED between Cube {i+1} and Cube {j+1}.")
                intersection_found = True

    # 2. If no intersections were found, proceed to calculate the bounding box
    print("\n2. Final Result:")
    if not intersection_found:
        print("   ✅ No intersections were found.")
        side_length = calculate_bounding_box_side(cubes)
        print(f"   📦 Side length of the smallest bounding cube: {side_length:.6f}")
    else:
        print("   ⚠️ Analysis stopped. Cannot calculate bounding box due to intersections.")
    print("--- Analysis Complete ---")



placements_no_intersection = np.array([[-8.77779155e-01, -8.77126384e-01, -1.01671182e+00,0,0,0], [-8.77779155e-01, -8.77126384e-01, 8.77819332e-01, 0,0,0], [-8.77779155e-01, 1.01740471e+00,-1.01671182e+00,0,0,0], [-8.77779155e-01, 1.01740471e+00,8.77819332e-01, 0,0,0], [ 1.01675167e+00,-8.77126384e-01, -1.01671182e+00,0,0,0], [ 1.01675167e+00,-8.77126384e-01, 8.77819332e-01, 0,0,0], [ 1.01675167e+00,1.01740471e+00,-1.01671182e+00,0,0,0], [ 1.01675167e+00,1.01740471e+00,8.77819332e-01, 0,0,0], [ 2.01013926e-01, 1.15252160e-01, 8.77806873e-01, 6.76458206e+00,8.99998576e+01, 1.60681931e+02], [ 1.13368230e-01, -8.77124005e-01, -2.08800374e-01, 2.07520455e+02, 3.59999904e+02, 9.00002316e+01], [-8.77752030e-01, 2.04448607e-01, -1.14126641e-01, 1.80000883e+02, 2.65856303e+01, 2.70001203e+02]])


analyze_cube_packing(placements_no_intersection)

import numpy as np
import plotly.graph_objects as go
import random

# --- Helper Functions (required for the visualization to run) ---
def rotation_matrix_3d(x_angle_deg, y_angle_deg, z_angle_deg):
    """Generates a 3D rotation matrix from Euler angles."""
    x_rad, y_rad, z_rad = np.deg2rad([x_angle_deg, y_angle_deg, z_angle_deg])
    cos_x, sin_x = np.cos(x_rad), np.sin(x_rad)
    cos_y, sin_y = np.cos(y_rad), np.sin(y_rad)
    cos_z, sin_z = np.cos(z_rad), np.sin(z_rad)
    rot_x = np.array([[1, 0, 0], [0, cos_x, -sin_x], [0, sin_x, cos_x]])
    rot_y = np.array([[cos_y, 0, sin_y], [0, 1, 0], [-sin_y, 0, cos_y]])
    rot_z = np.array([[cos_z, -sin_z, 0], [sin_z, cos_z, 0], [0, 0, 1]])
    return rot_z @ rot_y @ rot_x

def get_cube_vertices(center, rotation_matrix, side_length=1.0):
    """Calculates the world coordinates of a cube's 8 vertices."""
    half_side = side_length / 2.0
    local_verts = np.array([
        [-1, -1, -1], [-1, -1, 1], [-1, 1, -1], [-1, 1, 1],
        [1, -1, -1], [1, -1, 1], [1, 1, -1], [1, 1, 1]
    ]) * half_side
    return (local_verts @ rotation_matrix.T) + center


def get_random_color():
    """Generates a random hexadecimal color string."""
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)
    return f'#{r:02x}{g:02x}{b:02x}'

# --- Visualization Function using Plotly ---
def visualize_packing_3d(placements):
    """Visualizes a 3D cube packing as an interactive Plotly plot."""
    fig = go.Figure()

    # Define the 12 triangles that make up the faces of a cube
    # (2 triangles per face)
    # Indices correspond to the 8 vertices from get_cube_vertices
    face_triangles = np.array([
        [0, 1, 2], [1, 3, 2], # Back face
        [4, 6, 5], [5, 6, 7], # Front face
        [0, 4, 1], [1, 4, 5], # Bottom face
        [2, 3, 6], [3, 7, 6], # Top face
        [0, 2, 4], [2, 6, 4], # Left face
        [1, 5, 3], [3, 5, 7]  # Right face
    ])

    for i, placement in enumerate(placements):
        center = np.array(placement[:3])
        rotation_mat = rotation_matrix_3d(*placement[3:])
        vertices = get_cube_vertices(center, rotation_mat)

        fig.add_trace(go.Mesh3d(
            x=vertices[:, 0],
            y=vertices[:, 1],
            z=vertices[:, 2],
            i=face_triangles[:, 0],
            j=face_triangles[:, 1],
            k=face_triangles[:, 2],
            opacity=0.6,
            color=get_random_color(),
            name=f'Cube {i+1}'
        ))

    # Configure the plot layout
    fig.update_layout(
        title='3D Cube Packing (Interactive)',
        scene=dict(
            xaxis_title='X Axis',
            yaxis_title='Y Axis',
            zaxis_title='Z Axis',
            aspectmode='data'  # Ensures correct aspect ratio
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )

    fig.show()



visualize_packing_3d(placements_no_intersection)

**Prompt used**

Act as an expert in optimization and geometry, to solve the following question:
what is the smallest cube that can fit 'num_cubes' unit cubes in it without them
overlapping?

Your task is to produce a search function that searches for the best placements
of the num_cubes unit cubes. The side length of the big cube containing all num_cubes of
your unit cubes will be calculated automatically, and that will be your
score. More precisely, you will have to produce a list of placements, which is
num_cubes tuples of size 6 of the form (x, y, z, x_angle, y_angle, z_angle), and you will be evaluated with the
calculate_packing_score_cubes_3d(placements: List[Tuple], num_cubes: int) function which computes the smallest bounding box, and heavily penalizes overlapping unit cubes.
This function returns two things: the current score that we want to maximise, and also a "fixed" version of the input placements, which just fixes some obvious inaccuracies that may be present in the input.

You could call this as follows:
score, fixed_placements = calculate_packing_score_cubes_3d(placements, num_cubes)

You will also have access to a squeezing function that tries to greedily compress the existing configuration. You may call it for example like this:

squeezed_placements = squeeze_placements_3d([tuple(item) for item in placements], num_passes=5, binary_search_iterations=10)

This squeezing function usually improves things, but it can occasionally produce worse results than the input, its codebase is a bit unreliable.

You may code up any search method you want, and you are allowed to call the
calculate_packing_score_cubes_3d() function as many times as you want. You have
access to it, you don't need to code up the calculate_packing_score_cubes_3d() or the squeeze_placements_3d() functions. You want the score to be as high as possible!

Your task is to write a search function that searches for the best placements. Your
function will have 1000 seconds to run, and after that it has to have returned
the best construction it found. If after 1000 seconds it has not returned anything, the program will be terminated with a huge negative score.

Good luck!

In [ ]:
#@title Initial program

"""Finds the smallest cube into which n unit cubes can be packed."""
import itertools
import logging
import time
from scipy import integrate
import numpy as np
from scipy import optimize
import warnings
import random
import re
from collections.abc import Callable, Mapping
from typing import Any, List, Tuple, Dict
import scipy.linalg as la
import numpy.polynomial.polynomial as poly
import collections
import copy
import math



def scaling(input_arr, factor=1.001):
  new_arr = [list(item) for item in input_arr]
  for i in range(len(new_arr)):
    new_arr[i][0] *= factor
    new_arr[i][1] *= factor
    new_arr[i][2] *= factor
  return np.array(new_arr)


def search_for_best_packings_3d(num_cubes: int) -> List[Tuple]:
  """Searches for the best packing of unit cubes into a container cube."""
  variable_name = f'best_placements_3d_{num_cubes}'
  if np.random.rand() < 0.99 and variable_name in globals():
    placements = list(globals()[variable_name])
    # To avoid float point errors, we scale the placements by a small factor.
    placements = list(scaling(placements, factor=1.00001))
  else:
    side = math.ceil(num_cubes ** (1 / 3.0))
    placements = [
        (
            (np.random.rand() - 0.5) * side,
            (np.random.rand() - 0.5) * side,
            (np.random.rand() - 0.5) * side,
            np.random.uniform(0, 360),
            np.random.uniform(0, 360),
            np.random.uniform(0, 360),
        )
        for _ in range(num_cubes)
    ]

  best_placements = copy.deepcopy(placements)
  best_score, fixed_temp_placements = calculate_packing_score_cubes_3d(
      best_placements, num_cubes
  )
  temp_placements = [tuple(p) for p in fixed_temp_placements]
  print(
      f'Initial score: {best_score}, side:'
      f' {-best_score if best_score > -1000 else "N/A"}'
  )

  start_time = time.time()
  eval_count = 0
  running_time = np.random.randint(100, 1000)
  while time.time() - start_time < running_time:
    temp_placements = [list(p) for p in temp_placements]
    if np.random.rand() < 0.01:
      temp_placements = [list(p) for p in best_placements]
    idx_to_mutate = np.random.randint(0, num_cubes)

    temp_placements[idx_to_mutate][0] += np.random.normal(0, 0.01)
    temp_placements[idx_to_mutate][1] += np.random.normal(0, 0.01)
    temp_placements[idx_to_mutate][2] += np.random.normal(0, 0.01)
    for i in range(3, 6):
      temp_placements[idx_to_mutate][i] = (
          temp_placements[idx_to_mutate][i] + np.random.normal(0, 2)
      ) % 360
    candidate_placements = [tuple(p) for p in temp_placements]

    if eval_count % 100 == 0 and eval_count > 0 and best_score > -100:
      squeezed_candidate = squeeze_placements_3d(
          [tuple(item) for item in candidate_placements],
          num_passes=3,
          binary_search_iterations=10,
      )
      if len(squeezed_candidate) == num_cubes:
        candidate_placements = squeezed_candidate

    score, fixed_candidate_placements = calculate_packing_score_cubes_3d(
        candidate_placements, num_cubes
    )
    candidate_placements = [tuple(p) for p in fixed_candidate_placements]
    eval_count += 1

    if score > best_score:
      best_score = score
      best_placements = copy.deepcopy(candidate_placements)
      temp_placements = copy.deepcopy(candidate_placements)
      logging.info(
          f'New best score: {best_score}, side: {-score}, evals: {eval_count}'
      )
      squeezed_candidate = squeeze_placements_3d(
          [tuple(item) for item in best_placements],
          num_passes=10,
          binary_search_iterations=20,
      )
      score_after, fixed_squeezed_candidates = calculate_packing_score_cubes_3d(
          squeezed_candidate, num_cubes
      )
      logging.info(f'Score after squeeze: {score_after}')
      if score_after > best_score:
        best_score = score_after
        best_placements = copy.deepcopy(fixed_squeezed_candidates)
        temp_placements = copy.deepcopy(fixed_squeezed_candidates)
    if np.random.rand() < 0.01:
      temp_placements = [list(p) for p in best_placements]

  logging.info(f'Final score: {best_score}, side: {-best_score}')
  logging.info(f'Total Evaluations: {eval_count}')
  return best_placements




In [ ]:
#@title Evaluation function used

NUM_CUBES_IN_CORNERS = 8


# --- 3D GEOMETRY HELPER FUNCTIONS ---


def rotation_matrix_3d(x_angle_deg, y_angle_deg, z_angle_deg):
  """Generates a 3D rotation matrix from Euler angles (XYZ convention)."""
  x_angle_rad = np.deg2rad(x_angle_deg)
  y_angle_rad = np.deg2rad(y_angle_deg)
  z_angle_rad = np.deg2rad(z_angle_deg)
  cos_x, sin_x = np.cos(x_angle_rad), np.sin(x_angle_rad)
  cos_y, sin_y = np.cos(y_angle_rad), np.sin(y_angle_rad)
  cos_z, sin_z = np.cos(z_angle_rad), np.sin(z_angle_rad)

  rotation_x = np.array([[1, 0, 0], [0, cos_x, -sin_x], [0, sin_x, cos_x]])
  rotation_y = np.array([[cos_y, 0, sin_y], [0, 1, 0], [-sin_y, 0, cos_y]])
  rotation_z = np.array([[cos_z, -sin_z, 0], [sin_z, cos_z, 0], [0, 0, 1]])

  return rotation_z @ rotation_y @ rotation_x


def get_cube_vertices(center, rotation_matrix, side_length=1.0):
  """Calculates the world coordinates of a cube's vertices."""
  half_side = side_length / 2.0
  local_vertices = np.array([
      [-half_side, -half_side, -half_side],
      [-half_side, -half_side, half_side],
      [-half_side, half_side, -half_side],
      [-half_side, half_side, half_side],
      [half_side, -half_side, -half_side],
      [half_side, -half_side, half_side],
      [half_side, half_side, -half_side],
      [half_side, half_side, half_side],
  ])
  return (local_vertices @ rotation_matrix.T) + center


def get_cube_axes(rotation_matrix):
  """Calculates the axes to test for SAT, which are the face normals."""
  local_axes = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
  return local_axes @ rotation_matrix.T


def project_cube_onto_axis(vertices, axis):
  """Projects a cube's vertices onto an axis."""
  projections = vertices @ axis
  return np.min(projections), np.max(projections)


def do_cubes_intersect(vertices1, vertices2, axes1, axes2):
  """Checks for intersection between two cubes using the full Separating Axis Theorem."""
  # Combine the 6 face-normal axes from both cubes
  axes_to_test = list(axes1) + list(axes2)

  # **CORRECTION**: Add the 9 axes from the cross products of the cubes' axes.
  # This is the part that was missing from the original.
  for i in range(3):
    for j in range(3):
      cross_product = np.cross(axes1[i], axes2[j])
      # Add the axis only if it's not a zero vector (avoids parallel axes)
      if np.linalg.norm(cross_product) > 1e-6:
        axes_to_test.append(cross_product)

  # Test for separation along all 15 (or fewer) axes
  for axis in axes_to_test:
    min1, max1 = project_cube_onto_axis(vertices1, axis)
    min2, max2 = project_cube_onto_axis(vertices2, axis)

    # If there is no overlap on any axis, the cubes do not intersect
    if max1 < min2 - 1e-9 or max2 < min1 - 1e-9:
      return False  # Found a separating axis

  # If no separating axis was found after checking all potential axes,
  # the cubes must be intersecting.
  return True


def min_bounding_cube_extent_3d(all_vertices):
  """Calculates the side length of the minimum axis-aligned bounding cube."""
  if not all_vertices:
    return 0, np.array([0, 0, 0]), np.array([0, 0, 0])
  all_vertices_np = np.vstack(all_vertices)
  min_coords = np.min(all_vertices_np, axis=0)
  max_coords = np.max(all_vertices_np, axis=0)
  side_lengths = max_coords - min_coords
  return np.max(side_lengths), min_coords, max_coords


# --- SQUEEZING FUNCTION ---
def squeeze_placements_3d(
    placements: List[Tuple],
    num_passes: int = 3,
    binary_search_iterations: int = 10,
):
  """Adjusts cube placements to pack them more tightly, with special handling

  for corner and inner cubes.
  """
  if not placements:
    return []
  # print(f'Squeezing {len(placements)} cubes...')
  # print(placements)
  num_cubes = len(placements)
  if num_cubes <= 1:
    return placements

  current_placements = np.array([list(p) for p in placements])

  final_placements = [list(p) for p in current_placements]

  # Determine the bounding box of the initial "free" cubes
  _, min_b, max_b = min_bounding_cube_extent_3d([
      get_cube_vertices(p[:3], rotation_matrix_3d(p[3], p[4], p[5]))
      for p in final_placements
  ])

  # Set the positions for the corner cubes
  num_corner_cubes = min(num_cubes, NUM_CUBES_IN_CORNERS)
  if num_corner_cubes > 0:
    corner_centers = [
        np.array([min_b[0] + 0.5, min_b[1] + 0.5, min_b[2] + 0.5]),
        np.array([min_b[0] + 0.5, min_b[1] + 0.5, max_b[2] - 0.5]),
        np.array([min_b[0] + 0.5, max_b[1] - 0.5, min_b[2] + 0.5]),
        np.array([min_b[0] + 0.5, max_b[1] - 0.5, max_b[2] - 0.5]),
        np.array([max_b[0] - 0.5, min_b[1] + 0.5, min_b[2] + 0.5]),
        np.array([max_b[0] - 0.5, min_b[1] + 0.5, max_b[2] - 0.5]),
        np.array([max_b[0] - 0.5, max_b[1] - 0.5, min_b[2] + 0.5]),
        np.array([max_b[0] - 0.5, max_b[1] - 0.5, max_b[2] - 0.5]),
    ]
    for i in range(num_corner_cubes):
      final_placements[i][:3] = corner_centers[i]
      final_placements[i][3:] = [0, 0, 0]

  current_placements = np.array(final_placements)

  # Part 1: Squeeze the inner cubes (last num_cubes - NUM_CUBES_IN_CORNERS)
  # This part pushes these cubes towards the center, one axis at a time.
  made_change_in_global_pass = True
  for pass_idx in range(num_passes):
    if not made_change_in_global_pass:
      break
    made_change_in_global_pass = False

    for axis_idx in range(3):
      rotation_matrices = [
          rotation_matrix_3d(p[3], p[4], p[5]) for p in current_placements
      ]
      all_vertices = [
          get_cube_vertices(p[:3], rot)
          for p, rot in zip(current_placements, rotation_matrices)
      ]
      all_axes = [get_cube_axes(rot) for rot in rotation_matrices]
      avg_center = np.mean(current_placements[:, :3], axis=0)

      # Only iterate over the inner cubes for this type of squeezing
      inner_cube_indices = list(range(NUM_CUBES_IN_CORNERS, num_cubes))
      random.shuffle(inner_cube_indices)

      for i in inner_cube_indices:
        original_center = current_placements[i, :3].copy()
        vec_to_avg_center = avg_center - original_center

        move_vec = np.zeros(3)
        if abs(vec_to_avg_center[axis_idx]) < 1e-7:
          continue
        move_vec[axis_idx] = vec_to_avg_center[axis_idx]

        low_bound, high_bound = 0.0, 1.0
        best_non_colliding_factor = 0.0
        rot_i = rotation_matrices[i]
        axes_i = all_axes[i]

        for _ in range(binary_search_iterations):
          test_factor = (low_bound + high_bound) / 2.0
          test_center = original_center + test_factor * move_vec
          moved_vertices_i = get_cube_vertices(test_center, rot_i)

          collision = False
          for j in range(num_cubes):
            if i == j:
              continue
            if do_cubes_intersect(
                moved_vertices_i, all_vertices[j], axes_i, all_axes[j]
            ):
              collision = True
              break
          if collision:
            high_bound = test_factor
          else:
            best_non_colliding_factor = test_factor
            low_bound = test_factor

        if best_non_colliding_factor > 1e-5:
          displacement = best_non_colliding_factor * move_vec
          current_placements[i, :3] += displacement
          all_vertices[i] = get_cube_vertices(current_placements[i, :3], rot_i)
          made_change_in_global_pass = True

  # Part 2: Squeeze the corner cubes (first NUM_CUBES_IN_CORNERS)
  # This part finds a single factor `alpha` to push all corner cubes
  # simultaneously towards the center.
  if num_cubes > NUM_CUBES_IN_CORNERS > 0:
    rotation_matrices = [
        rotation_matrix_3d(p[3], p[4], p[5]) for p in current_placements
    ]
    all_vertices = [
        get_cube_vertices(p[:3], rot)
        for p, rot in zip(current_placements, rotation_matrices)
    ]
    all_axes = [get_cube_axes(rot) for rot in rotation_matrices]
    avg_center = np.mean(current_placements[:, :3], axis=0)

    corner_vectors = [
        avg_center - current_placements[i, :3]
        for i in range(NUM_CUBES_IN_CORNERS)
    ]

    low_alpha, high_alpha = 0.0, 1.0
    best_alpha = 0.0

    for _ in range(binary_search_iterations):
      test_alpha = (low_alpha + high_alpha) / 2.0
      if test_alpha == 0.0:
        continue

      collision_found = False
      for i in range(NUM_CUBES_IN_CORNERS):
        test_center_i = (
            current_placements[i, :3] + test_alpha * corner_vectors[i]
        )
        moved_vertices_i = get_cube_vertices(
            test_center_i, rotation_matrices[i]
        )

        # Check for intersection only against the inner cubes
        for j in range(NUM_CUBES_IN_CORNERS, num_cubes):
          if do_cubes_intersect(
              moved_vertices_i, all_vertices[j], all_axes[i], all_axes[j]
          ):
            collision_found = True
            break
        if collision_found:
          break

      if collision_found:
        high_alpha = test_alpha
      else:
        best_alpha = test_alpha
        low_alpha = test_alpha

    # Apply the best simultaneous move to all corner cubes
    if best_alpha > 1e-5:
      for i in range(NUM_CUBES_IN_CORNERS):
        current_placements[i, :3] += (best_alpha - 1e-9) * corner_vectors[i]
    # print(f"Best alpha: {best_alpha}")

  return [tuple(p) for p in current_placements]


def calculate_packing_score_cubes_3d(
    placements: List[Tuple],
    num_cubes: int,
    intersection_penalty_factor=100.0,
    return_adjusted_coordinates=True,
    verbose=False,
):
  """Calculates the score for a 3D cube packing configuration."""
  if not placements or len(placements) != num_cubes:
    return -1_000_000.0

  try:
    # Create a working copy of placements to modify
    final_placements = [list(p) for p in placements]

    # Determine the bounding box of the initial "free" cubes
    _, min_b, max_b = min_bounding_cube_extent_3d([
        get_cube_vertices(p[:3], rotation_matrix_3d(p[3], p[4], p[5]))
        for p in final_placements
    ])

    if verbose:
      print(f'Initial bounding box: {min_b}, {max_b}')

    # Set the positions for the corner cubes
    num_corner_cubes = min(num_cubes, NUM_CUBES_IN_CORNERS)
    if num_corner_cubes > 0:
      corner_centers = [
          np.array([min_b[0] + 0.5, min_b[1] + 0.5, min_b[2] + 0.5]),
          np.array([min_b[0] + 0.5, min_b[1] + 0.5, max_b[2] - 0.5]),
          np.array([min_b[0] + 0.5, max_b[1] - 0.5, min_b[2] + 0.5]),
          np.array([min_b[0] + 0.5, max_b[1] - 0.5, max_b[2] - 0.5]),
          np.array([max_b[0] - 0.5, min_b[1] + 0.5, min_b[2] + 0.5]),
          np.array([max_b[0] - 0.5, min_b[1] + 0.5, max_b[2] - 0.5]),
          np.array([max_b[0] - 0.5, max_b[1] - 0.5, min_b[2] + 0.5]),
          np.array([max_b[0] - 0.5, max_b[1] - 0.5, max_b[2] - 0.5]),
      ]
      for i in range(num_corner_cubes):
        final_placements[i][:3] = corner_centers[i]
        final_placements[i][3:] = [0, 0, 0]

    # NOW, calculate the definitive geometry based on the final placements
    rotation_matrices = [
        rotation_matrix_3d(p[3], p[4], p[5]) for p in final_placements
    ]
    all_vertices = [
        get_cube_vertices(p[:3], rot)
        for p, rot in zip(final_placements, rotation_matrices)
    ]
    all_axes = [get_cube_axes(rot) for rot in rotation_matrices]

    # The REAL side length is from the final, arranged configuration
    final_side_length, _, _ = min_bounding_cube_extent_3d(all_vertices)

    # Check for intersections
    intersection_count = 0
    for i in range(num_cubes):
      for j in range(i + 1, num_cubes):
        if do_cubes_intersect(
            all_vertices[i], all_vertices[j], all_axes[i], all_axes[j]
        ):
          intersection_count += 1
          if verbose:
            print(f'Intersection between cubes {i} and {j}')
            print(f'Vertices: {all_vertices[i]},\n {all_vertices[j]}')
            print(f'Axes: {all_axes[i]}, {all_axes[j]}')
            print(f'Minimum bounding box: {final_side_length}')

    score = (
        -final_side_length - intersection_count * intersection_penalty_factor
    )
    if return_adjusted_coordinates:
      return score, final_placements
    return score

  except (ValueError, IndexError) as e:
    return -1_000_002.0

In [ ]:
#@title Final code evolved by AlphaEvolve

"""Finds the smallest cube into which n unit cubes can be packed."""
import itertools
import logging
import time
from scipy import integrate
import numpy as np
from scipy import optimize
import warnings
import random
import re
from collections.abc import Callable, Mapping
from typing import Any, List, Tuple, Dict
import scipy.linalg as la
import numpy.polynomial.polynomial as poly
import collections
import copy
import math
from scipy.spatial.transform import Rotation as R # Import for rotations
import scipy.linalg # For null_space if needed, or QR decomposition


def normalize_angle(angle: float) -> float:
    """Normalizes an angle to be within [0, 360)."""
    angle = angle % 360.0
    if angle < 0:
        angle += 360.0
    return angle

# Angles relevant for cube packing (axis-aligned, 45-degree, and body-diagonal alignments)
THETA_BODY_DIAG = np.degrees(np.arctan(1/np.sqrt(2))) # approx 35.264 degrees

# Generate all relevant angles systematically
BASE_SNAP_ANGLES = np.array([
    0.0, 45.0, THETA_BODY_DIAG, 90.0 - THETA_BODY_DIAG
])

# Add 90, 180, 270 degrees to each base angle for full coverage
SNAP_ANGLES_CUBE = np.concatenate([
    BASE_SNAP_ANGLES + i for i in [0.0, 90.0, 180.0, 270.0]
])
SNAP_ANGLES_CUBE = np.unique(np.sort(SNAP_ANGLES_CUBE % 360.0))


# Helper to apply a global rotation and translation to a cube's (pos, angles)
def apply_global_transformation_to_cube(x: float, y: float, z: float, ax: float, ay: float, az: float,
                                        global_rot_matrix: R, translation_offset: np.ndarray) -> Tuple[float, float, float, float, float, float]:
    """
    Applies a global rotation and translation to a cube's position and orientation.
    Args:
        x, y, z: Current position of the cube's center.
        ax, ay, az: Current Euler angles (degrees) of the cube.
        global_rot_matrix: A scipy.spatial.transform.Rotation object representing the global rotation.
        translation_offset: A numpy array [tx, ty, tz] for the global translation.
    Returns:
        Tuple[float, float, float, float, float, float]: New (x, y, z, ax, ay, az) for the cube.
    """
    # Convert position to numpy array
    pos_vec = np.array([x, y, z])

    # Apply global rotation to position
    rotated_pos_vec = global_rot_matrix.apply(pos_vec)

    # Apply translation offset
    final_pos_vec = rotated_pos_vec + translation_offset

    # Original rotation of the cube
    R_orig = R.from_euler('xyz', [ax, ay, az], degrees=True)

    # Combine original rotation with global rotation.
    # The order global_rot_matrix * R_orig means global_rot_matrix is applied first in the sequence,
    # then R_orig relative to the newly rotated frame.
    R_new = global_rot_matrix * R_orig

    # Convert back to Euler angles
    new_ax, new_ay, new_az = R_new.as_euler('xyz', degrees=True)

    # Ensure angles are within [0, 360)
    # Normalize angles
    new_ax = normalize_angle(new_ax)
    new_ay = normalize_angle(new_ay)
    new_az = normalize_angle(new_az)

    return final_pos_vec[0], final_pos_vec[1], final_pos_vec[2], new_ax, new_ay, new_az

# Predefined canonical global rotations (Euler angles ZYX for consistency, but stored as XYZ for R.from_euler)
# These are key orientations for a single cube (or a collection) that may lead to dense packings.
CANONICAL_GLOBAL_ROTATIONS_EULER_XYZ = [
    (0.0, 0.0, 0.0), # Axis-aligned
    (0.0, 0.0, 45.0), # Z-axis 45-degree rotation
    (0.0, 45.0, 0.0), # Y-axis 45-degree rotation
    (45.0, 0.0, 0.0), # X-axis 45-degree rotation
    (90.0, 0.0, 0.0), # 90 degree X
    (0.0, 90.0, 0.0), # 90 degree Y
    (0.0, 0.0, 90.0), # 90 degree Z
]
# Ensure angles are within [0, 360) and keep only unique (though these are mostly unique)
CANONICAL_GLOBAL_ROTATIONS_EULER_XYZ_UNIQUE = []
for rot_tuple in CANONICAL_GLOBAL_ROTATIONS_EULER_XYZ:
    normalized_rot = tuple(a % 360 for a in rot_tuple)
    if not any(np.allclose(normalized_rot, existing_rot, atol=1e-3) for existing_rot in CANONICAL_GLOBAL_ROTATIONS_EULER_XYZ_UNIQUE):
        CANONICAL_GLOBAL_ROTATIONS_EULER_XYZ_UNIQUE.append(normalized_rot)
CANONICAL_GLOBAL_ROTATIONS_EULER_XYZ = CANONICAL_GLOBAL_ROTATIONS_EULER_XYZ_UNIQUE

# Motifs for "Crystalline Lattice Mutation" and predefined initial placements.
# Each motif is a list of (x, y, z, ax, ay, az) tuples, centered at origin.
# These are known optimal or highly dense packings for small numbers of cubes.
MOTIFS = {
    1: [
        (0.0, 0.0, 0.0, 0.0, 0.0, 0.0) # Single cube
    ],
    2: [ # Optimal for 2 cubes: two interlocked cubes (side length sqrt(2) approx 1.414)
        (0.0, 0.0, 0.0, 45.0, 0.0, 45.0),
        (0.0, 0.0, 0.0, 45.0, 90.0, 45.0)
    ],
    3: [ # Near-optimal for 3 cubes (can achieve side length ~1.83)
        (-0.5, 0.0, 0.0, 0.0, 0.0, 0.0),
        (0.5, 0.0, 0.0, 0.0, 0.0, 0.0),
        (0.0, 0.0, 0.0, 45.0, 45.0, 45.0) # One central rotated cube, two axis-aligned.
    ],
    4: [ # Optimal for 4 cubes: 2x2x1 axis-aligned block (side length 2.0)
        (-0.5, -0.5, 0.0, 0.0, 0.0, 0.0), (0.5, -0.5, 0.0, 0.0, 0.0, 0.0),
        (-0.5, 0.5, 0.0, 0.0, 0.0, 0.0), (0.5, 0.5, 0.0, 0.0, 0.0, 0.0)
    ],
    5: [ # This is a common optimal pattern for N=5, side length 2.0.
        (-0.5, -0.5, -0.5, 0.0, 0.0, 0.0), (0.5, -0.5, -0.5, 0.0, 0.0, 0.0),
        (-0.5, 0.5, -0.5, 0.0, 0.0, 0.0), (0.5, 0.5, -0.5, 0.0, 0.0, 0.0),
        (0.0, 0.0, 0.5, 0.0, 0.0, 0.0)
    ],
    6: [ # 2x2x2 minus 2, side 2.0
        (-0.5, -0.5, -0.5, 0.0, 0.0, 0.0), (0.5, -0.5, -0.5, 0.0, 0.0, 0.0),
        (-0.5, 0.5, -0.5, 0.0, 0.0, 0.0), (0.5, 0.5, -0.5, 0.0, 0.0, 0.0),
        (-0.5, -0.5, 0.5, 0.0, 0.0, 0.0), (0.5, 0.5, 0.5, 0.0, 0.0, 0.0),
    ],
    7: [ # 2x2x2 minus 1, side 2.0
        (-0.5, -0.5, -0.5, 0.0, 0.0, 0.0), (0.5, -0.5, -0.5, 0.0, 0.0, 0.0),
        (-0.5, 0.5, -0.5, 0.0, 0.0, 0.0), (0.5, 0.5, -0.5, 0.0, 0.0, 0.0),
        (-0.5, -0.5, 0.5, 0.0, 0.0, 0.0), (0.5, -0.5, 0.5, 0.0, 0.0, 0.0),
        (-0.5, 0.5, 0.5, 0.0, 0.0, 0.0)
    ],
    8: [ # Optimal for 8 cubes: 2x2x2 axis-aligned block (side length 2.0)
        (-0.5, -0.5, -0.5, 0.0, 0.0, 0.0), (0.5, -0.5, -0.5, 0.0, 0.0, 0.0),
        (-0.5, 0.5, -0.5, 0.0, 0.0, 0.0), (0.5, 0.5, -0.5, 0.0, 0.0, 0.0),
        (-0.5, -0.5, 0.5, 0.0, 0.0, 0.0), (0.5, -0.5, 0.5, 0.0, 0.0, 0.0),
        (-0.5, 0.5, 0.5, 0.0, 0.0, 0.0), (0.5, 0.5, 0.5, 0.0, 0.0, 0.0)
    ],
    9: [ # Optimal for 9 cubes: 8 cubes forming a 2x2x2 block, 1 central rotated cube (side length 2.0)
        (-0.5, -0.5, -0.5, 0.0, 0.0, 0.0), (0.5, -0.5, -0.5, 0.0, 0.0, 0.0),
        (-0.5, 0.5, -0.5, 0.0, 0.0, 0.0), (0.5, 0.5, -0.5, 0.0, 0.0, 0.0),
        (-0.5, -0.5, 0.5, 0.0, 0.0, 0.0), (0.5, -0.5, 0.5, 0.0, 0.0, 0.0),
        (-0.5, 0.5, 0.5, 0.0, 0.0, 0.0), (0.5, 0.5, 0.5, 0.0, 0.0, 0.0),
        (0.0, 0.0, 0.0, 45.0, 45.0, 45.0) # Central cube, rotated 45 degrees on all axes
    ]
}

def get_predefined_placements_for_packing(num_cubes: int) -> Optional[List[Tuple]]:
    """
    Returns a predefined optimal or near-optimal placement for specific num_cubes,
    or None if no such pattern is defined.
    """
    if num_cubes in MOTIFS:
        placements = list(MOTIFS[num_cubes]) # Make a copy to avoid modifying original motif
        # Add a very small noise to positions and angles to help SA fine-tune from here.
        pos_noise_std = 0.00001
        angle_noise_std = 0.001
        for i in range(len(placements)):
            p = list(placements[i])
            p[0] += np.random.normal(0, pos_noise_std)
            p[1] += np.random.normal(0, pos_noise_std)
            p[2] += np.random.normal(0, pos_noise_std)
            p[3] = normalize_angle(p[3] + np.random.normal(0, angle_noise_std))
            p[4] = normalize_angle(p[4] + np.random.normal(0, angle_noise_std))
            p[5] = normalize_angle(p[5] + np.random.normal(0, angle_noise_std))
            placements[i] = tuple(p)
        return placements
    return None


def _get_compact_grid_dimensions(num_cubes: int) -> Tuple[int, int, int]:
    """Calculates dimensions L, W, H for a compact rectangular grid for num_cubes."""
    if num_cubes <= 0:
        return 0, 0, 0

    best_L, best_W, best_H = num_cubes, 1, 1 # Default for 1D case, ensures volume is at least num_cubes
    min_max_dim = num_cubes # Minimize the maximum dimension
    min_sum_dim = num_cubes + 2 # Minimize the sum of dimensions (as a tie-breaker for max_dim)

    # A more restricted search space for L, W, H based on typical packing bounds
    # Max dimension is usually <= 2 * ceil(N^(1/3))
    max_dim_search = int(math.ceil(num_cubes**(1/3.0)) * 2) + 1 # Add 1 for safety to include max
    if num_cubes <= 8: max_dim_search = 3 # For small num_cubes, keep search very tight

    for L_candidate in range(1, min(num_cubes + 1, max_dim_search + 1)):
        for W_candidate in range(1, min(L_candidate + 1, max_dim_search + 1)): # W <= L for symmetry
            # Calculate H such that L*W*H >= num_cubes
            H_candidate = int(math.ceil(num_cubes / (L_candidate * W_candidate)))
            if H_candidate == 0: H_candidate = 1 # Ensure H is at least 1
            if H_candidate > max_dim_search: continue # Prune if H is already too large

            if L_candidate * W_candidate * H_candidate >= num_cubes:
                current_max_dim = max(L_candidate, W_candidate, H_candidate)
                current_sum_dim = L_candidate + W_candidate + H_candidate

                # Prioritize minimizing the maximum dimension
                if current_max_dim < min_max_dim:
                    min_max_dim = current_max_dim
                    min_sum_dim = current_sum_dim
                    best_L, best_W, best_H = L_candidate, W_candidate, H_candidate
                # If max dimensions are equal, prefer smaller sum of dimensions (more "cuboid" shape)
                elif current_max_dim == min_max_dim:
                    if current_sum_dim < min_sum_dim:
                        min_sum_dim = current_sum_dim
                        best_L, best_W, best_H = L_candidate, W_candidate, H_candidate

    return max(1, best_L), max(1, best_W), max(1, best_H)


def _apply_strategic_initial_angles(placements: List[List[float]]):
  """Applies strategic initial angles to placements, biasing towards common optimal orientations."""
  axis_aligned_prob = 0.4   # 40% chance for angles near 0, 90, 180, 270
  forty_five_deg_prob = 0.2 # 20% chance for angles near 45, 135, 225, 315
  body_diag_prob = 0.2 # 20% chance for angles near body diagonal angles
  # Remaining 20% will be fully random (1.0 - 0.4 - 0.2 - 0.2 = 0.2)

  angle_perturb_std = 5.0 # Small standard deviation for initial angle noise (in degrees)

  for p in placements:
    rand_initial_angle_strategy_selector = np.random.rand()

    if rand_initial_angle_strategy_selector < axis_aligned_prob:
        # Bias towards axis-aligned orientations
        base_angles_axial = [0.0, 90.0, 180.0, 270.0]
        p[3] = normalize_angle(np.random.choice(base_angles_axial) + np.random.normal(0.0, angle_perturb_std))
        p[4] = normalize_angle(np.random.choice(base_angles_axial) + np.random.normal(0.0, angle_perturb_std))
        p[5] = normalize_angle(np.random.choice(base_angles_axial) + np.random.normal(0.0, angle_perturb_std))
    elif rand_initial_angle_strategy_selector < axis_aligned_prob + forty_five_deg_prob:
        # Bias towards 45-degree rotations (important for some dense packings)
        base_angles_45_deg = [45.0, 135.0, 225.0, 315.0]
        p[3] = normalize_angle(np.random.choice(base_angles_45_deg) + np.random.normal(0.0, angle_perturb_std))
        p[4] = normalize_angle(np.random.choice(base_angles_45_deg) + np.random.normal(0.0, angle_perturb_std))
        p[5] = normalize_angle(np.random.choice(base_angles_45_deg) + np.random.normal(0.0, angle_perturb_std))
    elif rand_initial_angle_strategy_selector < axis_aligned_prob + forty_five_deg_prob + body_diag_prob:
        # Bias towards body-diagonal rotations
        # Use a subset of SNAP_ANGLES_CUBE that corresponds to body-diagonal angles
        body_diag_angles = [a for a in SNAP_ANGLES_CUBE if not np.isclose(a % 45.0, 0.0)]
        if not body_diag_angles: body_diag_angles = [35.26, 54.73] # Fallback if SNAP_ANGLES_CUBE is empty
        p[3] = normalize_angle(np.random.choice(body_diag_angles) + np.random.normal(0.0, angle_perturb_std))
        p[4] = normalize_angle(np.random.choice(body_diag_angles) + np.random.normal(0.0, angle_perturb_std))
        p[5] = normalize_angle(np.random.choice(body_diag_angles) + np.random.normal(0.0, angle_perturb_std))
    else:
        # Fully random angles for broad exploration
        p[3] = normalize_angle(np.random.uniform(0.0, 360.0))
        p[4] = normalize_angle(np.random.uniform(0.0, 360.0))
        p[5] = normalize_angle(np.random.uniform(0.0, 360.0))


def get_centroid(placements: List[Tuple]) -> Tuple[float, float, float]:
  """Calculates the centroid of a list of cube placements."""
  if not placements:
    return (0.0, 0.0, 0.0)
  x_coords = [p[0] for p in placements]
  y_coords = [p[1] for p in placements]
  z_coords = [p[2] for p in placements]
  return (sum(x_coords) / len(placements),
          sum(y_coords) / len(placements),
          sum(z_coords) / len(placements))

def recenter_placements(placements: List[Tuple]) -> List[Tuple]:
  """Recalculates the centroid of current placements and shifts them to be centered at origin."""
  if not placements:
    return []

  # Calculate centroid
  x_coords = [p[0] for p in placements]
  y_coords = [p[1] for p in placements]
  z_coords = [p[2] for p in placements]

  centroid_x = sum(x_coords) / len(placements)
  centroid_y = sum(y_coords) / len(placements)
  centroid_z = sum(z_coords) / len(placements)

  centered_placements = []
  for p in placements:
    centered_placements.append(
        (p[0] - centroid_x, p[1] - centroid_y, p[2] - centroid_z, p[3], p[4], p[5])
    )
  return centered_placements

def snap_angle_to_nearest_optimal(angle: float) -> float:
    """Snaps an angle to the nearest optimal packing angle (0, 45, 90, ...)."""
    angle = normalize_angle(angle) # Ensure angle is in [0, 360)

    # SNAP_ANGLES_CUBE is a global numpy array
    distances = np.abs(SNAP_ANGLES_CUBE - angle)
    # Handle wrap-around for angles near 0/360 degrees
    wrap_around_distances = np.abs(SNAP_ANGLES_CUBE - (angle - 360.0))
    distances = np.minimum(distances, wrap_around_distances)

    closest_idx = np.argmin(distances)
    return SNAP_ANGLES_CUBE[closest_idx]

def generate_centered_grid_placements(num_cubes: int) -> List[Tuple]:
  """Generates a compact rectangular grid of axis-aligned unit cubes centered around the origin."""
  if num_cubes <= 0:
      return []

  L, W, H = _get_compact_grid_dimensions(num_cubes)

  # Calculate offset to center the cluster of cubes around the origin.
  offset_x = (L - 1) / 2.0
  offset_y = (W - 1) / 2.0
  offset_z = (H - 1) / 2.0

  placements = []
  # Iterate through the calculated L, W, H dimensions to fill the rectangular prism
  for z_idx in range(H):
    for y_idx in range(W):
      for x_idx in range(L):
        if len(placements) < num_cubes: # Only add up to num_cubes
          # Apply centering offset and set rotations to 0.0 for axis-aligned cubes
          placements.append(
              (x_idx - offset_x, y_idx - offset_y, z_idx - offset_z, 0.0, 0.0, 0.0)
          )
        else:
          return placements # Return early once num_cubes are placed
  return placements

def generate_density_gradient_placements(num_cubes: int) -> List[Tuple]:
    """
    Generates an initial placement of num_cubes unit cubes where rotation
    angles are dependent on their distance from the center of the grid,
    aiming for a denser, more complex core and more axis-aligned outer cubes.
    """
    if num_cubes <= 0:
        return []

    # Start with a basic centered grid to get initial positions
    initial_grid_placements = generate_centered_grid_placements(num_cubes)
    if not initial_grid_placements:
        return []

    placements = []

    # Calculate distances and max_distance to normalize
    distances = []
    for p in initial_grid_placements:
        dist = np.linalg.norm(np.array(p[:3]))
        distances.append(dist)

    max_distance_overall = max(distances) if distances else 1.0 # Avoid division by zero, min 1.0

    pos_noise_std = 0.0001 # Keep position noise low
    angle_noise_std = 0.1 # Small noise for angles

    # Pre-calculate common rotation types
    # Body-diagonal set: (THETA_BODY_DIAG, THETA_BODY_DIAG, THETA_BODY_DIAG) permutations and their complements/90-deg rotations.
    # For simplicity, use the specific known optimal (45,45,45) and THETA_BODY_DIAG values.
    body_diag_angles = [
        (45.0, 45.0, 45.0),
        (THETA_BODY_DIAG, THETA_BODY_DIAG, THETA_BODY_DIAG),
        (90.0 - THETA_BODY_DIAG, 90.0 - THETA_BODY_DIAG, 90.0 - THETA_BODY_DIAG)
    ]
    # Face-diagonal set: one axis 45/90/etc, others 0/90.
    face_diag_angles = [
        (0.0, 0.0, 45.0), (0.0, 45.0, 0.0), (45.0, 0.0, 0.0),
        (0.0, 0.0, 90.0), (0.0, 90.0, 0.0), (90.0, 0.0, 0.0),
        (0.0, 0.0, 135.0), (0.0, 135.0, 0.0), (135.0, 0.0, 0.0)
    ]
    # Axis-aligned set: all 0 or 90 or 180 or 270.
    axis_aligned_angles = [
        (0.0, 0.0, 0.0), (90.0, 0.0, 0.0), (0.0, 90.0, 0.0), (0.0, 0.0, 90.0),
        (90.0, 90.0, 0.0), (90.0, 0.0, 90.0), (0.0, 90.0, 90.0), (90.0, 90.0, 90.0)
    ]

    for i, p in enumerate(initial_grid_placements):
        cx, cy, cz = p[:3]
        distance = distances[i]
        normalized_distance = distance / max_distance_overall # 0.0 at center, 1.0 at max_distance

        ax, ay, az = 0.0, 0.0, 0.0 # Default fallback

        # Define radial zones for angle assignments
        if normalized_distance < 0.25: # Innermost core (e.g., central 25% of cubes by distance)
            # Prioritize body-diagonal type rotations
            chosen_angles = random.choice(body_diag_angles)
            ax, ay, az = chosen_angles
            angle_perturb_factor = 0.5 # Larger noise for more exploration in the core

        elif normalized_distance < 0.65: # Mid-range (between 25% and 65%)
            # Prioritize face-diagonal type rotations
            chosen_angles = random.choice(face_diag_angles)
            ax, ay, az = chosen_angles
            angle_perturb_factor = 0.3 # Medium noise

        else: # Outermost layer (beyond 65%)
            # Prioritize axis-aligned rotations
            chosen_angles = random.choice(axis_aligned_angles)
            ax, ay, az = chosen_angles
            angle_perturb_factor = 0.1 # Smallest noise for stability at outer boundaries

        # Add a perturbation to the chosen angles for fine-tuning by SA
        ax = normalize_angle(ax + np.random.normal(0, angle_noise_std * angle_perturb_factor))
        ay = normalize_angle(ay + np.random.normal(0, angle_noise_std * angle_perturb_factor))
        az = normalize_angle(az + np.random.normal(0, angle_noise_std * angle_perturb_factor))

        # Keep positions from the initial grid, add very tiny noise
        final_cx = cx + np.random.normal(0, pos_noise_std)
        final_cy = cy + np.random.normal(0, pos_noise_std)
        final_cz = cz + np.random.normal(0, pos_noise_std)

        placements.append((final_cx, final_cy, final_cz, ax, ay, az))

    return placements


def generate_staggered_placements(num_cubes: int) -> List[Tuple]:
    """Generates a compact rectangular grid of axis-aligned unit cubes,
    with alternating layers offset, centered around the origin.
    This aims to provide a denser initial packing than a simple grid."""
    if num_cubes <= 0:
        return []

    L, W, H = _get_compact_grid_dimensions(num_cubes)

    placements = []
    # Staggering logic: offset alternate layers in X and Y by 0.5 units
    for z_idx in range(H):
      for y_idx in range(W):
        for x_idx in range(L):
          if len(placements) < num_cubes:
            x_offset = 0.0
            y_offset = 0.0
            # Apply offset to odd layers (z_idx is 0-indexed)
            if (z_idx % 2) == 1:
                x_offset = 0.5
                y_offset = 0.5

            placements.append(
                (x_idx + x_offset, y_idx + y_offset, z_idx, 0.0, 0.0, 0.0)
            )
          else:
            break # Stop adding cubes once num_cubes is reached
        if len(placements) >= num_cubes:
            break
      if len(placements) >= num_cubes:
          break

    # Calculate overall centroid of the generated points to center them.
    # This accounts for the actual bounds of the staggered arrangement.
    max_x = max(p[0] for p in placements) if placements else 0.0
    min_x = min(p[0] for p in placements) if placements else 0.0
    max_y = max(p[1] for p in placements) if placements else 0.0
    min_y = min(p[1] for p in placements) if placements else 0.0
    max_z = max(p[2] for p in placements) if placements else 0.0
    min_z = min(p[2] for p in placements) if placements else 0.0

    actual_center_x = (max_x + min_x) / 2.0
    actual_center_y = (max_y + min_y) / 2.0
    actual_center_z = (max_z + min_z) / 2.0

    centered_placements = []
    for p in placements:
        centered_placements.append(
            (p[0] - actual_center_x, p[1] - actual_center_y, p[2] - actual_center_z, p[3], p[4], p[5])
        )

    return centered_placements

def generate_crystalline_checkerboard_placements(num_cubes: int) -> List[Tuple]:
    """
    Generates an initial placement of num_cubes unit cubes in a compact grid,
    with an alternating 'checkerboard' pattern of axis-aligned (0,0,0)
    and diagonally rotated (45,45,45) cubes. This aims to create an interlocked,
    dense initial structure.
    """
    if num_cubes <= 0:
        return []

    L, W, H = _get_compact_grid_dimensions(num_cubes)

    placements = []

    # Calculate offset to center the cluster of cubes around the origin.
    offset_x = (L - 1) / 2.0
    offset_y = (W - 1) / 2.0
    offset_z = (H - 1) / 2.0

    pos_noise_std = 0.0001
    angle_noise_std = 0.01

    for z_idx in range(H):
        for y_idx in range(W):
            for x_idx in range(L):
                if len(placements) < num_cubes:
                    cx = (x_idx - offset_x) + np.random.normal(0, pos_noise_std)
                    cy = (y_idx - offset_y) + np.random.normal(0, pos_noise_std)
                    cz = (z_idx - offset_z) + np.random.normal(0, pos_noise_std)

                    # Apply checkerboard rotation pattern
                    if (x_idx + y_idx + z_idx) % 2 == 0: # Even sum of indices
                        ax, ay, az = (0.0, 0.0, 0.0) # Axis-aligned
                    else: # Odd sum of indices
                        ax, ay, az = (45.0, 45.0, 45.0) # Diagonally rotated

                    ax = normalize_angle(ax + np.random.normal(0, angle_noise_std))
                    ay = normalize_angle(ay + np.random.normal(0, angle_noise_std))
                    az = normalize_angle(az + np.random.normal(0, angle_noise_std))

                    placements.append((cx, cy, cz, ax, ay, az))
                else:
                    return placements
    return placements

def search_for_best_packings_3d(num_cubes: int) -> List[Tuple]:
  """Searches for the best packing of unit cubes into a container cube."""
  variable_name = f'best_placements_3d_{num_cubes}'

  initial_placements_tuple = []
  eval_count = 0

  # Option 1: Try to load previous best from globals()
  if variable_name in globals() and np.random.rand() < 0.99:
    candidate_placements_tuple = [tuple(p) for p in globals()[variable_name]]
    score, fixed_candidate = calculate_packing_score_cubes_3d(candidate_placements_tuple, num_cubes)
    eval_count += 1
    if score > float('-inf'): # Ensure it's a valid placement
        initial_placements_tuple = fixed_candidate
        best_score = score
    else: # Fallback to generating if loaded configuration is invalid
        logging.warning(f"Loaded configuration for {num_cubes} cubes was invalid. Generating new.")

  # Option 2: If no valid loaded configuration, generate and evaluate initial strategies
  if not initial_placements_tuple or best_score == float('-inf'):
    # List of initial placement strategies
    initial_strategies_funcs = [
        get_predefined_placements_for_packing,
        generate_crystalline_checkerboard_placements, # New strategy: Crystalline Checkerboard
        generate_centered_grid_placements,
        generate_staggered_placements,
        generate_density_gradient_placements,
    ]

    current_best_initial_score = -float('inf')
    current_best_initial_placements = []

    for generate_func in initial_strategies_funcs:
        if num_cubes == 0: continue

        temp_placements_list_tuples = generate_func(num_cubes)
        if temp_placements_list_tuples is None: # For "Predefined" strategy, it might return None
            continue

        # Convert to list of lists for potential modification
        temp_placements_list_mutable = [list(p) for p in temp_placements_list_tuples]

        # Apply additional angle strategies only if the generator didn't already handle complex angles
        # Apply additional angle strategies only if the generator didn't already handle complex angles.
        # Functions that produce their own angles: get_predefined_placements_for_packing,
        # generate_density_gradient_placements, and generate_crystalline_checkerboard_placements.
        # Functions that start with (0,0,0) angles: generate_centered_grid_placements, generate_staggered_placements.
        if generate_func in [generate_centered_grid_placements, generate_staggered_placements]:
            _apply_strategic_initial_angles(temp_placements_list_mutable)

        strategy_name = generate_func.__name__ # Use function name for logging
        logging.info(f'Evaluating initial configuration: {strategy_name} for num_cubes={num_cubes}')

        score, fixed_placements = calculate_packing_score_cubes_3d(temp_placements_list_mutable, num_cubes)
        eval_count += 1

        squeezed_placements = squeeze_placements_3d(
            [tuple(item) for item in fixed_placements],
            num_passes=20, # Aggressive squeeze for initial configurations
            binary_search_iterations=40,
        )
        squeezed_score, fixed_squeezed = calculate_packing_score_cubes_3d(squeezed_placements, num_cubes)
        eval_count += 1

        if squeezed_score > current_best_initial_score:
            current_best_initial_score = squeezed_score
            current_best_initial_placements = copy.deepcopy(fixed_squeezed)
            logging.info(f'New best initial from {strategy_name}: {current_best_initial_score}, side: {-current_best_initial_score}')

    initial_placements_tuple = current_best_initial_placements
    best_score = current_best_initial_score

    # Handle case where num_cubes is 0 or initial generation fails somehow
    if not initial_placements_tuple or best_score == float('-inf'):
        initial_placements_tuple = [(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)] * num_cubes
        if num_cubes > 0:
            logging.warning("Initial placement generation failed or was invalid. Using fallback to default origin cube(s).")
        score_fallback, fixed_fallback = calculate_packing_score_cubes_3d(
            initial_placements_tuple, num_cubes
        )
        eval_count += 1
        initial_placements_tuple = fixed_fallback
        best_score = score_fallback

  current_score, fixed_current_placements = calculate_packing_score_cubes_3d(
      initial_placements_tuple, num_cubes # initial_placements_tuple is already prepared (loaded or generated)
  )
  current_placements_tuple = [tuple(p) for p in fixed_current_placements]
  eval_count += 1 # Count initial score evaluation

  best_placements = copy.deepcopy(current_placements_tuple)
  best_score = current_score

  print(
      f'Initial score: {best_score}, side:'
      f' {-best_score if best_score > -1000 else "N/A"}'
  )

  start_time = time.time()
  # eval_count is already initialized and incremented in the initial placement block.
  last_best_score_update_eval = eval_count # Initialize stagnation tracker with initial evaluations
  max_running_time = 990  # Use almost full allocated time for the search

  # SA temperature parameters (Adjusted for simpler mutation scheme)
  initial_temperature = 1.0 # Higher initial temperature for more aggressive exploration
  min_temperature = 1e-7 # Standard minimum temperature

  # SA temperature parameters
  initial_temperature = 1.5 # Start hotter for more exploration
  min_temperature = 1e-8 # Slightly lower min temp

  # Parameters for individual_perturbation, local_correlated_perturbation, correlated_subset_perturbation
  base_pos_std_initial = 0.20 # Slightly larger initial positional noise
  base_pos_std_min = 0.000005 # Slightly smaller final positional noise
  base_angle_std_initial = 30.0 # Slightly larger initial angular noise
  base_angle_std_min = 0.0005 # Slightly smaller final angular noise

  # Parameters for Global Rotation Mutation (kept separate due to larger range)
  global_rot_std_initial = 90.0 # Degrees, focus on more common global rotations
  global_rot_std_min = 0.5 # Degrees

  # Parameters for Slip Plane Mutation
  slip_magnitude_std_initial = 0.30 # Units of distance
  slip_magnitude_std_min = 0.001 # Units of distance

  # Parameters for Rigid Sub-Cluster Transformation
  sub_cluster_rigid_pos_std_initial = 0.5 # Larger initial position shift for rigid clusters
  sub_cluster_rigid_pos_std_min = 0.005 # Final position shift for rigid clusters
  sub_cluster_rigid_angle_std_initial = 45.0 # Larger initial angle for rigid clusters
  sub_cluster_rigid_angle_std_min = 0.5 # Final angle for rigid clusters

  # Parameters for Periodic Squeeze of current SA state
  initial_periodic_squeeze_prob = 0.05 # Start lower for more exploration
  final_periodic_squeeze_prob = 0.20 # End higher for more exploitation via squeeze

  initial_periodic_squeezing_passes = 3 # Start with fewer passes
  final_periodic_squeezing_passes = 15 # End with more thorough passes

  initial_periodic_squeezing_iterations = 5 # Start with fewer iterations
  final_periodic_squeezing_iterations = 30 # End with more thorough iterations

  # Mutation probabilities (initial and final for annealing)
  # Sum of these probabilities should be 1.0.
  mutation_probs_initial = {
      'revert_to_best': 0.001, # Renamed from full_reset
      'global_scaling': 0.02,
      'global_rotation': 0.03,
      'snap_to_angle': 0.05666,
      'crystalline_lattice_mutation': 0.14,
      'rigid_sub_cluster_transformation': 0.08,
      'local_correlated_perturbation': 0.35,
      'individual_perturbation': 0.15,
      'cohesive_force_mutation': 0.08,
      'stochastic_relocation_mutation': 0.04, # Reduced slightly
      'layer_shift_mutation': 0.03, # Reduced slightly
      'quantum_leap_mutation': 0.01, # New mutation, low initial probability
      'crystalline_shear_mutation': 0.03, # New mutation, medium initial probability
  }
  mutation_probs_final = {
      'revert_to_best': 0.0005,
      'global_scaling': 0.005,
      'global_rotation': 0.002,
      'snap_to_angle': 0.23,
      'crystalline_lattice_mutation': 0.06,
      'rigid_sub_cluster_transformation': 0.04,
      'local_correlated_perturbation': 0.38,
      'individual_perturbation': 0.10,
      'cohesive_force_mutation': 0.14,
      'stochastic_relocation_mutation': 0.005, # Reduced more
      'layer_shift_mutation': 0.025, # Reduced more
      'quantum_leap_mutation': 0.005, # Kept low, more of a disruptor
      'crystalline_shear_mutation': 0.06, # Increased in final stages for exploitation
  }

  while time.time() - start_time < max_running_time:
    elapsed_time_ratio = (time.time() - start_time) / max_running_time
    elapsed_time_ratio = min(max(elapsed_time_ratio, 0.0), 1.0) # Clamp

    # Annealing schedule for temperature
    current_T = initial_temperature * (min_temperature / initial_temperature) ** elapsed_time_ratio
    current_T = max(min_temperature, current_T)

    # --- Adaptive Reheating Logic ---
    # If a certain number of evaluations have passed without improvement, reheat.
    stagnation_threshold_evals = 1000 + 50 * num_cubes # Adjust threshold dynamically, more for larger N
    if eval_count - last_best_score_update_eval > stagnation_threshold_evals:
        # Reheat temperature to a fraction of initial temperature or current T.
        reheat_factor = 0.5 # Reheat to 50% of initial temperature or a higher fraction of current T
        current_T = max(initial_temperature * reheat_factor, current_T * 1.5) # Reheat or increase by 1.5x, whichever is higher
        logging.info(f"Reheating SA: score stagnated for {stagnation_threshold_evals} evals. New Temp: {current_T:.4f}")
        last_best_score_update_eval = eval_count # Reset stagnation tracker after reheating

    # Anneal step sizes for perturbations
    pos_std = base_pos_std_initial * (base_pos_std_min / base_pos_std_initial) ** elapsed_time_ratio
    pos_std = max(base_pos_std_min, pos_std)
    angle_std = base_angle_std_initial * (base_angle_std_min / base_angle_std_initial) ** elapsed_time_ratio
    angle_std = max(base_angle_std_min, angle_std)

    # Anneal global rotation std
    global_rot_std = global_rot_std_initial * (global_rot_std_min / global_rot_std_initial) ** elapsed_time_ratio
    global_rot_std = max(global_rot_std_min, global_rot_std)

    # Anneal slip magnitude std
    slip_magnitude_std = slip_magnitude_std_initial * (slip_magnitude_std_min / slip_magnitude_std_initial) ** elapsed_time_ratio
    slip_magnitude_std = max(slip_magnitude_std_min, slip_magnitude_std)

    # Anneal rigid sub-cluster transformation stds
    sub_cluster_rigid_pos_std = sub_cluster_rigid_pos_std_initial * (sub_cluster_rigid_pos_std_min / sub_cluster_rigid_pos_std_initial) ** elapsed_time_ratio
    sub_cluster_rigid_pos_std = max(sub_cluster_rigid_pos_std_min, sub_cluster_rigid_pos_std)
    sub_cluster_rigid_angle_std = sub_cluster_rigid_angle_std_initial * (sub_cluster_rigid_angle_std_min / sub_cluster_rigid_angle_std_initial) ** elapsed_time_ratio
    sub_cluster_rigid_angle_std = max(sub_cluster_rigid_angle_std_min, sub_cluster_rigid_angle_std)

    # Anneal periodic squeeze probability and passes/iterations
    periodic_squeeze_prob = initial_periodic_squeeze_prob * (final_periodic_squeeze_prob / initial_periodic_squeeze_prob) ** (1 - elapsed_time_ratio)
    periodic_squeezing_passes_current = int(initial_periodic_squeezing_passes + (final_periodic_squeezing_passes - initial_periodic_squeezing_passes) * elapsed_time_ratio)
    periodic_squeezing_iterations_current = int(initial_periodic_squeezing_iterations + (final_periodic_squeezing_iterations - initial_periodic_squeezing_iterations) * elapsed_time_ratio)

    # PERIODIC SQUEEZE ON CURRENT SA STATE (Greedy Local Optimization)
    if np.random.rand() < periodic_squeeze_prob:
        logging.info(f"Applying periodic squeeze to current SA state. Temp: {current_T:.4f}, Passes: {periodic_squeezing_passes_current}, Iterations: {periodic_squeezing_iterations_current}")
        squeezed_sa_state = squeeze_placements_3d(
            current_placements_tuple, # Use the current SA state
            num_passes=periodic_squeezing_passes_current,
            binary_search_iterations=periodic_squeezing_iterations_current,
        )
        squeezed_sa_score, fixed_squeezed_sa_state = calculate_packing_score_cubes_3d(squeezed_sa_state, num_cubes)
        eval_count += 1 # Count this evaluation

        # If the squeezed state is an improvement, update current SA state.
        if squeezed_sa_score > current_score:
            current_score = squeezed_sa_score
            current_placements_tuple = fixed_squeezed_sa_state
            logging.info(f"Periodic squeeze improved current SA state to score: {current_score}, side: {-current_score}")
            if current_score > best_score:
                best_score = current_score
                best_placements = copy.deepcopy(current_placements_tuple)
                logging.info(f'New absolute best score (from periodic squeeze): {best_score}, side: {-best_score if best_score > -1000 else "N/A"}')


    candidate_placements_list = [list(p) for p in current_placements_tuple] # Start from current SA state

    # Anneal mutation probabilities
    current_mutation_probs = {}
    for key in mutation_probs_initial:
        initial_p = mutation_probs_initial[key]
        final_p = mutation_probs_final[key]
        # Linear interpolation of probabilities: initial_p at ratio=0, final_p at ratio=1
        current_mutation_probs[key] = initial_p + (final_p - initial_p) * elapsed_time_ratio

    # Normalize probabilities to ensure they sum to 1
    sum_current_probs = sum(current_mutation_probs.values())
    if sum_current_probs > 0: # Avoid division by zero
        for key in current_mutation_probs:
            current_mutation_probs[key] /= sum_current_probs
    else: # Fallback to equal probabilities if sum is zero (should not happen with current defs)
        num_mutations = len(mutation_probs_initial)
        for key in mutation_probs_initial:
            current_mutation_probs[key] = 1.0 / num_mutations

    mutation_types = list(current_mutation_probs.keys())
    mutation_probabilities = list(current_mutation_probs.values())

    # Determine which mutation to apply
    mutation_choice = np.random.choice(mutation_types, p=mutation_probabilities)

    if mutation_choice == 'revert_to_best': # Renamed from full_reset
        candidate_placements_list = [list(p) for p in best_placements]
        logging.info(f"Reverted to best placements.")
    elif mutation_choice == 'global_scaling':
        # Apply scaling relative to the current centroid of the cubes
        current_centroid = get_centroid([tuple(p) for p in candidate_placements_list])
        # scale_factor varies less aggressively than pos_std
        scale_perturb_std = (base_pos_std_initial * 0.1) * ((base_pos_std_min * 0.1) / (base_pos_std_initial * 0.1)) ** elapsed_time_ratio
        scale_factor = 1.0 + np.random.normal(0, scale_perturb_std)
        scale_factor = np.clip(scale_factor, 0.95, 1.05) # Prevent extreme scaling
        for i in range(num_cubes):
            candidate_placements_list[i][0] = current_centroid[0] + (candidate_placements_list[i][0] - current_centroid[0]) * scale_factor
            candidate_placements_list[i][1] = current_centroid[1] + (candidate_placements_list[i][1] - current_centroid[1]) * scale_factor
            candidate_placements_list[i][2] = current_centroid[2] + (candidate_placements_list[i][2] - current_centroid[2]) * scale_factor
    elif mutation_choice == 'global_rotation':
        rot_x_global = np.random.normal(0, global_rot_std)
        rot_y_global = np.random.normal(0, global_rot_std)
        rot_z_global = np.random.normal(0, global_rot_std)
        global_rot_obj = R.from_euler('xyz', [rot_x_global, rot_y_global, rot_z_global], degrees=True)
        current_centroid = get_centroid([tuple(p) for p in candidate_placements_list])
        for i in range(num_cubes):
            # Rotate position relative to centroid
            pos = np.array(candidate_placements_list[i][:3]) - current_centroid
            rotated_pos = global_rot_obj.apply(pos)
            candidate_placements_list[i][0] = rotated_pos[0] + current_centroid[0]
            candidate_placements_list[i][1] = rotated_pos[1] + current_centroid[1]
            candidate_placements_list[i][2] = rotated_pos[2] + current_centroid[2]
            # Combine cube's own rotation with global rotation
            current_cube_rot = R.from_euler('xyz', candidate_placements_list[i][3:6], degrees=True)
            new_cube_rot_obj = global_rot_obj * current_cube_rot
            new_angles = new_cube_rot_obj.as_euler('xyz', degrees=True)
            candidate_placements_list[i][3] = normalize_angle(new_angles[0])
            candidate_placements_list[i][4] = normalize_angle(new_angles[1])
            candidate_placements_list[i][5] = normalize_angle(new_angles[2])
    elif mutation_choice == 'snap_to_angle':
        if num_cubes == 0: continue
        idx_to_mutate = np.random.randint(0, num_cubes)
        # Apply small position perturbation for shaking
        candidate_placements_list[idx_to_mutate][0] += np.random.normal(0, pos_std * 0.1)
        candidate_placements_list[idx_to_mutate][1] += np.random.normal(0, pos_std * 0.1)
        candidate_placements_list[idx_to_mutate][2] += np.random.normal(0, pos_std * 0.1)
        # Snap angles and add small perturbation
        for angle_idx in range(3, 6):
            snapped_angle = snap_angle_to_nearest_optimal(candidate_placements_list[idx_to_mutate][angle_idx])
            candidate_placements_list[idx_to_mutate][angle_idx] = normalize_angle(snapped_angle + np.random.normal(0, angle_std * 0.05))
    elif mutation_choice == 'correlated_subset_perturbation': # Formerly 'cluster_mutation'
        if num_cubes < 2: continue
        # Cluster size adapts: larger clusters early, smaller for fine-tuning
        max_cluster_size = max(1, int(num_cubes * (0.6 - 0.5 * elapsed_time_ratio))) # Max 0.6*N, min 0.1*N
        min_cluster_size = 1
        cluster_size = np.random.randint(min_cluster_size, max_cluster_size + 1)

        cluster_indices = np.random.choice(num_cubes, cluster_size, replace=False)

        trans_x = np.random.normal(0, pos_std * 2.0) # Larger step for clusters
        trans_y = np.random.normal(0, pos_std * 2.0)
        trans_z = np.random.normal(0, pos_std * 2.0)
        rot_x = np.random.normal(0, angle_std * 1.5) # Larger angle for clusters
        rot_y = np.random.normal(0, angle_std * 1.5)
        rot_z = np.random.normal(0, angle_std * 1.5)

        for idx in cluster_indices:
            candidate_placements_list[idx][0] += trans_x
            candidate_placements_list[idx][1] += trans_y
            candidate_placements_list[idx][2] += trans_z
            candidate_placements_list[idx][3] = normalize_angle(candidate_placements_list[idx][3] + rot_x)
            candidate_placements_list[idx][4] = normalize_angle(candidate_placements_list[idx][4] + rot_y)
            candidate_placements_list[idx][5] = normalize_angle(candidate_placements_list[idx][5] + rot_z)
    elif mutation_choice == 'crystalline_lattice_mutation':
        if num_cubes == 0: continue
        available_motif_sizes = [s for s in MOTIFS.keys() if s <= num_cubes]
        if not available_motif_sizes:
            logging.debug("No suitable motifs for Crystalline Lattice Mutation, falling back to local correlated.")
            # Fallback to local correlated perturbation if no motifs available for current num_cubes
            mutation_choice = 'local_correlated_perturbation'
        else:
            motif_size = np.random.choice(available_motif_sizes)
            chosen_motif = MOTIFS[motif_size]

            # Choose a strategic global rotation from the predefined list
            if CANONICAL_GLOBAL_ROTATIONS_EULER_XYZ:
                global_rot_euler = random.choice(CANONICAL_GLOBAL_ROTATIONS_EULER_XYZ)
                global_rot_matrix = R.from_euler('xyz', global_rot_euler, degrees=True)
                # Add a small perturbation to the strategic rotation to allow fine-tuning
                global_rot_noise_std = angle_std * 0.1 # Use annealed angle_std for noise
                global_rot_matrix = R.from_euler('xyz', np.array(global_rot_euler) + np.random.normal(0, global_rot_noise_std, 3), degrees=True)
            else: # Fallback to random if for some reason canonical rotations are empty
                global_rot_rand_euler = np.random.uniform(0, 360, 3)
                global_rot_matrix = R.from_euler('xyz', global_rot_rand_euler, degrees=True)

            # Apply a smaller perturbation factor when closer to optimal temperature
            # This applies to the noise added to motif positions and orientations
            perturb_factor = 0.05 + 0.1 * (1.0 - elapsed_time_ratio)
            perturb_factor = max(0.01, perturb_factor) # Minimum perturb factor

            if num_cubes == motif_size: # If motif size matches total cubes, replace all
                new_placements_from_motif = []
                for mx, my, mz, max_orig, may_orig, maz_orig in chosen_motif:
                    new_x, new_y, new_z, new_ax, new_ay, new_az = apply_global_transformation_to_cube(
                        mx, my, mz, max_orig, may_orig, maz_orig, global_rot_matrix, np.array([0.0,0.0,0.0])
                    )
                    # Add noise
                    new_x += np.random.normal(0, pos_std * perturb_factor)
                    new_y += np.random.normal(0, pos_std * perturb_factor)
                    new_z += np.random.normal(0, pos_std * perturb_factor)
                    new_ax = normalize_angle(new_ax + np.random.normal(0, angle_std * perturb_factor))
                    new_ay = normalize_angle(new_ay + np.random.normal(0, angle_std * perturb_factor))
                    new_az = normalize_angle(new_az + np.random.normal(0, angle_std * perturb_factor))
                    new_placements_from_motif.append([new_x, new_y, new_z, new_ax, new_ay, new_az])

                candidate_placements_list = recenter_placements([tuple(p) for p in new_placements_from_motif])
                candidate_placements_list = [list(p) for p in candidate_placements_list]

            else: # Replace a subset with the motif
                indices_to_replace = np.random.choice(num_cubes, motif_size, replace=False)

                selected_cubes_positions = np.array([
                    candidate_placements_list[i][:3] for i in indices_to_replace
                ])
                if len(selected_cubes_positions) == 0: continue
                current_subset_centroid = np.mean(selected_cubes_positions, axis=0)

                new_motif_placements = []
                for mx, my, mz, max_orig, may_orig, maz_orig in chosen_motif:
                    new_x, new_y, new_z, new_ax, new_ay, new_az = apply_global_transformation_to_cube(
                        mx, my, mz, max_orig, may_orig, maz_orig, global_rot_matrix, current_subset_centroid
                    )
                    # Add noise
                    new_x += np.random.normal(0, pos_std * perturb_factor)
                    new_y += np.random.normal(0, pos_std * perturb_factor)
                    new_z += np.random.normal(0, pos_std * perturb_factor)
                    new_ax = normalize_angle(new_ax + np.random.normal(0, angle_std * perturb_factor))
                    new_ay = normalize_angle(new_ay + np.random.normal(0, angle_std * perturb_factor))
                    new_az = normalize_angle(new_az + np.random.normal(0, angle_std * perturb_factor))
                    new_motif_placements.append([new_x, new_y, new_z, new_ax, new_ay, new_az])

                for i, original_idx in enumerate(indices_to_replace):
                    if i < len(new_motif_placements):
                       candidate_placements_list[original_idx] = new_motif_placements[i]
    elif mutation_choice == 'rigid_sub_cluster_transformation':
        if num_cubes < 2: continue # Needs at least 2 cubes for a cluster

        # Choose a cluster size between 2 and N/2, or at least 2
        cluster_size = np.random.randint(2, max(3, num_cubes // 2 + 1))
        cluster_indices = np.random.choice(num_cubes, cluster_size, replace=False)

        # Calculate centroid of the selected sub-cluster
        cluster_positions = np.array([candidate_placements_list[idx][:3] for idx in cluster_indices])
        if cluster_positions.shape[0] == 0: continue # Should not happen if cluster_size >= 1
        cluster_centroid = np.mean(cluster_positions, axis=0)

        # Generate translation for the rigid sub-cluster
        trans_offset = np.random.normal(0, sub_cluster_rigid_pos_std, 3)

        # Generate rotation for the rigid sub-cluster
        rot_x_cluster = np.random.normal(0, sub_cluster_rigid_angle_std)
        rot_y_cluster = np.random.normal(0, sub_cluster_rigid_angle_std)
        rot_z_cluster = np.random.normal(0, sub_cluster_rigid_angle_std)
        cluster_rot_obj = R.from_euler('xyz', [rot_x_cluster, rot_y_cluster, rot_z_cluster], degrees=True)

        for idx in cluster_indices:
            # Apply translation relative to cluster centroid, then add offset
            pos_vec = np.array(candidate_placements_list[idx][:3])
            pos_relative_to_centroid = pos_vec - cluster_centroid
            rotated_pos_relative = cluster_rot_obj.apply(pos_relative_to_centroid)
            final_pos = rotated_pos_relative + cluster_centroid + trans_offset

            candidate_placements_list[idx][0] = final_pos[0]
            candidate_placements_list[idx][1] = final_pos[1]
            candidate_placements_list[idx][2] = final_pos[2]

            # Combine current cube rotation with cluster rotation
            current_cube_rot = R.from_euler('xyz', candidate_placements_list[idx][3:6], degrees=True)
            new_cube_rot_obj = cluster_rot_obj * current_cube_rot # Apply cluster rot, then cube's own
            new_angles = new_cube_rot_obj.as_euler('xyz', degrees=True)
            candidate_placements_list[idx][3] = normalize_angle(new_angles[0])
            candidate_placements_list[idx][4] = normalize_angle(new_angles[1])
            candidate_placements_list[idx][5] = normalize_angle(new_angles[2])
        logging.debug(f"Rigid Sub-Cluster Transformation applied to {cluster_size} cubes.")

    elif mutation_choice == 'cohesive_force_mutation':
        if num_cubes == 0: continue
        current_centroid = get_centroid([tuple(p) for p in candidate_placements_list])
        for i in range(num_cubes):
            pos_vec = np.array(candidate_placements_list[i][:3])
            direction_vector = current_centroid - pos_vec
            dist = np.linalg.norm(direction_vector)
            if dist > 1e-9: # Avoid division by zero for cube at centroid
                normalized_direction = direction_vector / dist
                # Move magnitude scales with pos_std and also with current distance (for stronger pull on outliers)
                # Max 0.5 * pos_std to keep it gentle. Clamp to avoid overshooting.
                move_magnitude = np.random.normal(0, pos_std * 0.5)
                move_magnitude = np.clip(move_magnitude, -0.1, 0.1) # Prevent too large steps

                candidate_placements_list[i][0] += normalized_direction[0] * move_magnitude
                candidate_placements_list[i][1] += normalized_direction[1] * move_magnitude
                candidate_placements_list[i][2] += normalized_direction[2] * move_magnitude
        logging.debug("Cohesive Force (Simulated Gravity) mutation applied.")
    elif mutation_choice == 'stochastic_relocation_mutation':
        if num_cubes == 0: continue
        # Estimate current bounding box side length from best_score, or a fallback
        current_side_length_estimate = -best_score if best_score > -1000 else 2.0 * math.ceil(num_cubes**(1/3.0))
        relocation_range_half = current_side_length_estimate * 0.75 # Allow a bit beyond current bounds for exploration

        # Select 1 to N/4 cubes for aggressive relocation
        num_to_relocate = np.random.randint(1, max(2, num_cubes // 4 + 1))
        indices_to_relocate = np.random.choice(num_cubes, num_to_relocate, replace=False)

        for idx in indices_to_relocate:
            # Completely randomize position within the estimated bounding box range
            candidate_placements_list[idx][0] = np.random.uniform(-relocation_range_half, relocation_range_half)
            candidate_placements_list[idx][1] = np.random.uniform(-relocation_range_half, relocation_range_half)
            candidate_placements_list[idx][2] = np.random.uniform(-relocation_range_half, relocation_range_half)

            # Randomize angles or snap them to strategic ones with noise
            if np.random.rand() < 0.7: # 70% chance to pick a strategic angle with noise
                ax = normalize_angle(snap_angle_to_nearest_optimal(np.random.uniform(0, 360)) + np.random.normal(0, angle_std * 0.1))
                ay = normalize_angle(snap_angle_to_nearest_optimal(np.random.uniform(0, 360)) + np.random.normal(0, angle_std * 0.1))
                az = normalize_angle(snap_angle_to_nearest_optimal(np.random.uniform(0, 360)) + np.random.normal(0, angle_std * 0.1))
            else: # 30% chance for completely random angle
                ax = normalize_angle(np.random.uniform(0, 360))
                ay = normalize_angle(np.random.uniform(0, 360))
                az = normalize_angle(np.random.uniform(0, 360))

            candidate_placements_list[idx][3] = ax
            candidate_placements_list[idx][4] = ay
            candidate_placements_list[idx][5] = az
        logging.debug(f"Stochastic Relocation mutation applied to {num_to_relocate} cubes.")
    elif mutation_choice == 'layer_shift_mutation':
        if num_cubes < 2: continue # Needs at least 2 cubes for a layer shift

        axis = np.random.randint(0, 3) # 0:X, 1:Y, 2:Z

        # Find min/max for the chosen axis to determine a reasonable split point
        axis_coords = [p[axis] for p in candidate_placements_list]
        min_coord = min(axis_coords)
        max_coord = max(axis_coords)

        # Choose a random split point along the axis. Bias towards central splits.
        # Ensure split_point is not too close to min/max to ensure both sides have cubes.
        # A 0.1 buffer on min/max should be enough.
        if max_coord - min_coord < 0.2: # If spread is too small, no meaningful split
            split_point = (min_coord + max_coord) / 2.0
        else:
            split_point = np.random.uniform(min_coord + 0.1, max_coord - 0.1)

        # Determine the shift direction and magnitude
        # Shift can be along one of the two other axes, or along the same axis (compaction/expansion)
        possible_shift_axes = [0, 1, 2]
        shift_axis_idx = np.random.choice(possible_shift_axes)

        shift_magnitude = np.random.normal(0, pos_std * 0.75) # Use pos_std as base for magnitude
        shift_magnitude = np.clip(shift_magnitude, -0.2, 0.2) # Limit the shift magnitude to prevent huge jumps

        moved_cubes_count = 0
        for i in range(num_cubes):
            if candidate_placements_list[i][axis] > split_point:
                candidate_placements_list[i][shift_axis_idx] += shift_magnitude
                moved_cubes_count += 1

        if moved_cubes_count == 0 or moved_cubes_count == num_cubes:
            # If no cubes moved or all cubes moved, it means the split didn't create two meaningful groups.
            # While SA can filter this out, we can make it more explicit that this specific mutation
            # might not have been effective for this split. For simplicity, we just log and proceed.
            pass

        logging.debug(f"Layer Shift mutation applied along axis {axis} for {moved_cubes_count} cubes with shift {shift_magnitude:.3f} along axis {shift_axis_idx}.")
    elif mutation_choice == 'crystalline_shear_mutation':
        if num_cubes < 2: continue

        # 1. Choose a shear plane axis (X, Y, or Z)
        shear_axis_idx = np.random.randint(0, 3) # 0:X, 1:Y, 2:Z

        # 2. Determine a random split point along this axis
        axis_coords = np.array([p[shear_axis_idx] for p in candidate_placements_list])
        min_coord, max_coord = np.min(axis_coords), np.max(axis_coords)

        if max_coord - min_coord < 0.2: # If spread is too small for a meaningful split
            split_point = (min_coord + max_coord) / 2.0
        else:
            # Bias split point towards center, but still random. Use a normal dist around mean.
            mean_coord = np.mean(axis_coords)
            std_coord = np.std(axis_coords)
            # Ensure split point is within reasonable bounds
            split_point = np.random.normal(mean_coord, std_coord * 0.5)
            split_point = np.clip(split_point, min_coord - 0.5, max_coord + 0.5) # Allow some extension

        # 3. Determine the shear vector (translation) and shear rotation (around shear_axis_idx)
        # The shear translation is in the plane perpendicular to the shear_axis_idx

        other_axes = [i for i in [0, 1, 2] if i != shear_axis_idx]

        shear_trans_magnitude = np.random.normal(0, pos_std * 0.75) # Positional shift
        shear_trans_magnitude = np.clip(shear_trans_magnitude, -0.2, 0.2) # Limit magnitude

        # Shear translation vector is along one of the other axes, or a combination
        shear_vector = np.zeros(3)
        # Randomly choose one of the two perpendicular axes for the primary shift direction
        shear_vector[random.choice(other_axes)] = shear_trans_magnitude

        # Shear rotation: rotation around the shear_axis_idx
        shear_rot_angle = np.random.normal(0, angle_std * 0.5) # Angular shift
        shear_rot_angle = np.clip(shear_rot_angle, -10.0, 10.0) # Limit rotation magnitude

        # Create a rotation object for the shear rotation
        shear_rot_euler = np.zeros(3)
        shear_rot_euler[shear_axis_idx] = shear_rot_angle
        shear_rot_obj = R.from_euler('xyz', shear_rot_euler, degrees=True)

        # 4. Apply transformation to cubes on one side of the split plane
        moved_cubes_count = 0

        # Determine the "reference point" for rotation and translation, typically the split plane's center
        shear_reference_point = np.array([0.0, 0.0, 0.0])
        shear_reference_point[shear_axis_idx] = split_point

        for i in range(num_cubes):
            if candidate_placements_list[i][shear_axis_idx] > split_point: # Move cubes on one side
                # Translate position relative to the shear reference point, then rotate, then add back reference + shear vector
                pos_vec = np.array(candidate_placements_list[i][:3])
                pos_relative_to_ref = pos_vec - shear_reference_point
                rotated_pos_relative = shear_rot_obj.apply(pos_relative_to_ref)
                final_pos = rotated_pos_relative + shear_reference_point + shear_vector

                candidate_placements_list[i][0] = final_pos[0]
                candidate_placements_list[i][1] = final_pos[1]
                candidate_placements_list[i][2] = final_pos[2]

                # Combine cube's own rotation with shear rotation
                current_cube_rot = R.from_euler('xyz', candidate_placements_list[i][3:6], degrees=True)
                new_cube_rot_obj = shear_rot_obj * current_cube_rot
                new_angles = new_cube_rot_obj.as_euler('xyz', degrees=True)
                candidate_placements_list[i][3] = normalize_angle(new_angles[0])
                candidate_placements_list[i][4] = normalize_angle(new_angles[1])
                candidate_placements_list[i][5] = normalize_angle(new_angles[2])

                moved_cubes_count += 1

        logging.debug(f"Crystalline Shear mutation applied along axis {shear_axis_idx} for {moved_cubes_count} cubes.")
    elif mutation_choice == 'quantum_leap_mutation':
        if num_cubes == 0: continue

        # Select a random subset of cubes for the "quantum leap"
        # Number of cubes to affect: between 1 and N/3, but at least 1.
        num_to_affect = np.random.randint(1, max(2, num_cubes // 3 + 1))
        indices_to_affect = np.random.choice(num_cubes, num_to_affect, replace=False)

        # Get the centroid of the selected subset for rigid transformation
        subset_positions = np.array([candidate_placements_list[idx][:3] for idx in indices_to_affect])
        if len(subset_positions) == 0: continue
        subset_centroid = np.mean(subset_positions, axis=0)

        # Apply a random *global* transformation (translation + rotation) to the subset
        # This translation can be significant. Scale by an estimate of the current packing size.
        current_side_length_estimate = -best_score if best_score > -1000 else 2.0 * math.ceil(num_cubes**(1/3.0))
        # Larger translation for early stages, smaller later
        leap_pos_std = current_side_length_estimate * (0.5 - 0.4 * elapsed_time_ratio)
        leap_angle_std = 90.0 * (0.5 - 0.4 * elapsed_time_ratio) # Large angle shift

        trans_offset = np.random.normal(0, leap_pos_std, 3)
        rot_angles = np.random.normal(0, leap_angle_std, 3)
        leap_rot_obj = R.from_euler('xyz', rot_angles, degrees=True)

        for idx in indices_to_affect:
            # Apply rigid transformation
            pos_vec = np.array(candidate_placements_list[idx][:3])
            pos_relative_to_subset_centroid = pos_vec - subset_centroid
            rotated_pos_relative = leap_rot_obj.apply(pos_relative_to_subset_centroid)
            final_pos = rotated_pos_relative + subset_centroid + trans_offset

            candidate_placements_list[idx][0] = final_pos[0]
            candidate_placements_list[idx][1] = final_pos[1]
            candidate_placements_list[idx][2] = final_pos[2]

            # Re-project angles to be near optimal/strategic ones, with noise
            # Instead of just adding noise to current angles, re-initialize them strategically
            chosen_snap_angle_x = snap_angle_to_nearest_optimal(np.random.uniform(0, 360))
            chosen_snap_angle_y = snap_angle_to_nearest_optimal(np.random.uniform(0, 360))
            chosen_snap_angle_z = snap_angle_to_nearest_optimal(np.random.uniform(0, 360))

            # Apply a small amount of annealed angle_std noise
            candidate_placements_list[idx][3] = normalize_angle(chosen_snap_angle_x + np.random.normal(0, angle_std * 0.1))
            candidate_placements_list[idx][4] = normalize_angle(chosen_snap_angle_y + np.random.normal(0, angle_std * 0.1))
            candidate_placements_list[idx][5] = normalize_angle(chosen_snap_angle_z + np.random.normal(0, angle_std * 0.1))

        logging.debug(f"Quantum Leap mutation applied to {num_to_affect} cubes.")

    else: # 'individual_perturbation' is the fallback, but now it's explicit in probabilities
        # This branch should ideally not be hit if probabilities sum to 1 and all choices are covered.
        # But for safety, keep it as the last-resort default.
        if num_cubes == 0: continue
        idx_to_mutate = np.random.randint(0, num_cubes)
        candidate_placements_list[idx_to_mutate][0] += np.random.normal(0, pos_std)
        candidate_placements_list[idx_to_mutate][1] += np.random.normal(0, pos_std)
        candidate_placements_list[idx_to_mutate][2] += np.random.normal(0, pos_std)
        for i in range(3, 6):
            candidate_placements_list[idx_to_mutate][i] = normalize_angle(candidate_placements_list[idx_to_mutate][i] + np.random.normal(0, angle_std))

    # Ensure all angles are within [0, 360) after all mutations
    for i in range(num_cubes):
        for j in range(3, 6):
            candidate_placements_list[i][j] = normalize_angle(candidate_placements_list[i][j])

    # Convert to tuple list for scoring and squeezing
    candidate_placements_tuple = [tuple(p) for p in candidate_placements_list]

    candidate_score, fixed_candidate_placements = calculate_packing_score_cubes_3d(
        candidate_placements_tuple, num_cubes
    )
    candidate_placements_tuple = [tuple(p) for p in fixed_candidate_placements] # Use the fixed version
    eval_count += 1

    # Adaptive Squeezing on Candidate Placements
    # Only squeeze if the candidate is already performing well (i.e., better than current_score)
    # and occasionally, to give it a chance to improve further before acceptance.
    if candidate_score > current_score and np.random.rand() < 0.3: # ~30% chance if candidate is better
        squeezed_candidate_tuple = squeeze_placements_3d(
            [tuple(item) for item in candidate_placements_tuple],
            num_passes=2, # Lighter squeeze for promising candidates
            binary_search_iterations=5,
        )
        if len(squeezed_candidate_tuple) == num_cubes:
            squeezed_score, fixed_squeezed_candidate = calculate_packing_score_cubes_3d(
                squeezed_candidate_tuple, num_cubes
            )
            eval_count += 1 # Count this squeeze evaluation
            if squeezed_score > candidate_score: # Only use if squeeze actually improved it
                candidate_score = squeezed_score
                candidate_placements_tuple = [tuple(p) for p in fixed_squeezed_candidate]

    # --- Simulated Annealing (SA) Acceptance Criterion ---
    # T is already calculated as current_T

    accept_move = False
    if candidate_score > current_score: # Always accept improvements
        accept_move = True
    else: # Accept worse solutions with probability
        delta_score = candidate_score - current_score # This will be negative
        acceptance_prob = np.exp(delta_score / max(current_T, 1e-10)) # Use max with small epsilon to avoid division by zero
        accept_move = np.random.rand() < acceptance_prob

    if accept_move:
        current_score = candidate_score
        # The 'fixed_candidate_placements' from calculate_packing_score_cubes_3d are generally recentered.
        # So no explicit recentering is needed here.
        current_placements_tuple = copy.deepcopy(candidate_placements_tuple)

        # If the accepted solution is also the overall best found so far
        if current_score > best_score:
            best_score = current_score
            # best_placements should also be fixed by calculate_packing_score_cubes_3d
            best_placements = copy.deepcopy(current_placements_tuple)
            last_best_score_update_eval = eval_count # Update stagnation tracker
            logging.info(
                f'New best score: {best_score}, side: {-best_score if best_score > -1000 else "N/A"}, evals: {eval_count}, Temp: {current_T:.4f}'
            )
            # Aggressively squeeze the new best configuration, same intensity as initial strategies
            squeezed_best_tuple = squeeze_placements_3d(
                [tuple(item) for item in best_placements],
                num_passes=20, # Increased
                binary_search_iterations=40, # Increased
            )
            score_after_squeeze, fixed_squeezed_best = calculate_packing_score_cubes_3d(
                squeezed_best_tuple, num_cubes
            )
            eval_count += 1 # Count this evaluation too
            logging.info(f'Score after squeeze (on best): {score_after_squeeze}')
            if score_after_squeeze > best_score: # Only update if squeeze truly improved
                best_score = score_after_squeeze
                best_placements = copy.deepcopy(fixed_squeezed_best)
                current_placements_tuple = copy.deepcopy(fixed_squeezed_best)
                last_best_score_update_eval = eval_count # Update stagnation tracker again if squeeze improved

  logging.info(f'Final score: {best_score}, side: {-best_score}')
  logging.info(f'Total Evaluations: {eval_count}')
  return best_placements
